In [1]:
!pip install  peft trl==v0.11.2 bitsandbytes GPUtil transformers accelerate bitsandbytes optimum auto_gptq  \
    -U --no-index --find-links /kaggle/input/notebookaaf1a23696

!pip install --no-deps --no-index /kaggle/input/logits-processor-zoo/logits_processor_zoo-0.1.0-py3-none-any.whl

Looking in links: /kaggle/input/notebookaaf1a23696
Processing /kaggle/input/notebookaaf1a23696/peft-0.13.2-py3-none-any.whl
Processing /kaggle/input/notebookaaf1a23696/trl-0.11.2-py3-none-any.whl
Processing /kaggle/input/notebookaaf1a23696/bitsandbytes-0.44.1-py3-none-manylinux_2_24_x86_64.whl
Processing /kaggle/input/notebookaaf1a23696/GPUtil-1.4.0-py3-none-any.whl
Processing /kaggle/input/notebookaaf1a23696/transformers-4.46.3-py3-none-any.whl
Processing /kaggle/input/notebookaaf1a23696/accelerate-1.1.1-py3-none-any.whl
Processing /kaggle/input/notebookaaf1a23696/optimum-1.23.3-py3-none-any.whl
Processing /kaggle/input/notebookaaf1a23696/auto_gptq-0.7.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing /kaggle/input/notebookaaf1a23696/tyro-0.9.2-py3-none-any.whl (from trl==v0.11.2)
Processing /kaggle/input/notebookaaf1a23696/coloredlogs-15.0.1-py2.py3-none-any.whl (from optimum)
Processing /kaggle/input/notebookaaf1a23696/rouge-1.0.1-py3-none-any.whl (from auto_gp

In [2]:
!pip install --no-index --find-links=/kaggle/input/wheels-vllm-0-6-3-post1 torchvision==0.19.1
!pip install --no-index --find-links=/kaggle/input/wheels-vllm-0-6-3-post1 vllm

Looking in links: /kaggle/input/wheels-vllm-0-6-3-post1
Processing /kaggle/input/wheels-vllm-0-6-3-post1/torchvision-0.19.1-cp310-cp310-manylinux1_x86_64.whl
Processing /kaggle/input/wheels-vllm-0-6-3-post1/torch-2.4.1-cp310-cp310-manylinux1_x86_64.whl (from torchvision==0.19.1)
Processing /kaggle/input/wheels-vllm-0-6-3-post1/nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (from torch==2.4.1->torchvision==0.19.1)
Processing /kaggle/input/wheels-vllm-0-6-3-post1/nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (from torch==2.4.1->torchvision==0.19.1)
Processing /kaggle/input/wheels-vllm-0-6-3-post1/nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (from torch==2.4.1->torchvision==0.19.1)
Processing /kaggle/input/wheels-vllm-0-6-3-post1/nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl (from torch==2.4.1->torchvision==0.19.1)
Processing /kaggle/input/wheels-vllm-0-6-3-post1/nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl 

#### Retrieval using 7B LLM
Taken from: https://www.kaggle.com/code/sayoulala/use-llm-embedding-recall-infer

In [3]:
%%writefile run_recall.py
import pandas as pd

def preprocess_text(x):
    import re
    x = re.sub("http\w+", '',x)   # Delete URL
    x = re.sub(r"\.+", ".", x)    # Replace consecutive commas and periods with one comma and period character
    x = re.sub(r"\,+", ",", x)
    x = re.sub(r"\\\(", " ", x)
    x = re.sub(r"\\\)", " ", x)
    x = re.sub(r"[ ]{1,}", " ", x)
    x = x.strip()                 # Remove empty characters at the beginning and end
    return x

path_prefix = "/kaggle/input/eedi-mining-misconceptions-in-mathematics"
model_path = "/kaggle/input/qwenqwen2.5-7b-instruct/pytorch/qwen2.5-14b-instruct/1"
lora_path="/kaggle/input/sys-v1/sys-v1-checkpoint-4980"
device='cuda:0'

full_df = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/test.csv")

PROMPT=""""
SubjectName: {SubjectName}
ConstructName: {ConstructName}
QuestionText: {QuestionText}
Correct Answer: {CorrectAnswer}
Misconcepte Incorrect answer: {INcorrectAnswer}
"""

task_description = 'Given a math question and a misconcepte incorrect answer, please retrieve the most accurate reason for the misconception.'

def get_detailed_instruct(task_description: str, query: str) -> str:
    return f'Instruct: {task_description}\nQuery: {query}'


rows = []
for idx, row in full_df.iterrows():
    for option in ["A", "B", "C", "D"]:
        if option == row.CorrectAnswer:
            continue
            
        correct_answer = row[f"Answer{row.CorrectAnswer}Text"]

        #query_text =f"###question###:{row['SubjectName']}-{row['ConstructName']}-{row['QuestionText']}\n###Correct Answer###:{correct_answer}\n###Misconcepte Incorrect answer###:{option}.{row[f'Answer{option}Text']}"
        query_text = PROMPT.format(SubjectName=row['SubjectName'],ConstructName=row['ConstructName'],QuestionText=row['QuestionText'], CorrectAnswer=correct_answer, INcorrectAnswer=row[f'Answer{option}Text'])
        query_text = preprocess_text(query_text)
        row['query_text'] = get_detailed_instruct(task_description, query_text)

        rows.append({"query_text": query_text, 
                     "QuestionId_Answer": f"{row.QuestionId}_{option}",
                     "ConstructName": row.ConstructName,
                     "SubjectName": row.SubjectName,
                     "QuestionText": row.QuestionText,
                     "correct_answer": correct_answer,
                     "incorrect_answer": row[f"Answer{option}Text"],
                     "QuestionId": row['QuestionId'],
                     })

df = pd.DataFrame(rows)
df['order_index'] = list(range(len(df)))

import torch
from numpy.linalg import norm
import torch.nn.functional as F
from torch import Tensor
from transformers import AutoTokenizer, AutoModel,BitsAndBytesConfig
from peft import (
    LoraConfig,
    get_peft_model,
    PeftModel,
)

def batch_to_device(batch, target_device):
    """
    send a pytorch batch to a device (CPU/GPU)
    """
    for key in batch:
        if isinstance(batch[key], Tensor):
            batch[key] = batch[key].to(target_device)
    return batch

def last_token_pool(last_hidden_states: Tensor,
                    attention_mask: Tensor) -> Tensor:
    left_padding = (attention_mask[:, -1].sum() == attention_mask.shape[0])
    if left_padding:
        return last_hidden_states[:, -1]
    else:
        sequence_lengths = attention_mask.sum(dim=1) - 1
        batch_size = last_hidden_states.shape[0]
        return last_hidden_states[torch.arange(batch_size, device=last_hidden_states.device), sequence_lengths]

# def get_detailed_instruct(task_description: str, query: str) -> str:
#     return f'Instruct: {task_description}\nQuery: {query}'

def inference(df, model, tokenizer, device):
    batch_size = 16
    max_length = 512
    sentences = list(df['query_text'].values)

    all_embeddings = []
    length_sorted_idx = np.argsort([-len(sen) for sen in sentences])
    sentences_sorted = [sentences[idx] for idx in length_sorted_idx]
    for start_index in trange(0, len(sentences), batch_size, desc="Batches", disable=False):
        sentences_batch = sentences_sorted[start_index: start_index + batch_size]
        features = tokenizer(sentences_batch, max_length=max_length, padding=True, truncation=True,
                             return_tensors="pt")
        features = batch_to_device(features, model.device)
        with torch.no_grad():
            outputs = model.model(**features)
            embeddings = last_token_pool(outputs.last_hidden_state, features['attention_mask'])
            embeddings = torch.nn.functional.normalize(embeddings, dim=-1)
            embeddings = embeddings.detach().cpu().numpy().tolist()
        all_embeddings.extend(embeddings)

    all_embeddings = [np.array(all_embeddings[idx]).reshape(1, -1) for idx in np.argsort(length_sorted_idx)]

    return np.concatenate(all_embeddings, axis=0)


tokenizer = AutoTokenizer.from_pretrained(lora_path)
bnb_config = BitsAndBytesConfig(
            load_in_8bit=True,
            # bnb_4bit_use_double_quant=True,
            # bnb_4bit_quant_type="nf4",
            # bnb_4bit_compute_dtype=torch.bfloat16
        )
model = AutoModel.from_pretrained(model_path,
                                  quantization_config=bnb_config, 
                                  device_map="auto",
                                  )


# model = get_peft_model(model, config)
model = PeftModel.from_pretrained(model, lora_path)

model = model.eval()
# model = model.to(device)


import numpy as np
from tqdm.autonotebook import trange


task_description = 'Given a math question and a misconcepte incorrect answer, please retrieve the most accurate reason for the misconception.'


V_answer = inference(df, model, tokenizer, device)

misconception_df = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/misconception_mapping.csv")
if full_df.shape[0]<10:
    misconception_df = misconception_df.sample(n=40,random_state=2023)
misconception_df["query_text"] = misconception_df["MisconceptionName"]

V_misconception = inference(misconception_df, model, tokenizer, device)
print(V_misconception.shape)


from transformers import AutoConfig, AutoTokenizer, AutoModelForMaskedLM, AutoModel
import sys, os
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn.functional as F
import torch.nn as nn
import math
from sklearn.neighbors import NearestNeighbors


os.environ["TOKENIZERS_PARALLELISM"] = "false"

def get_matches(V_topic, V_content, n_neighbors=25):
    
    neighbors_model = NearestNeighbors(n_neighbors=n_neighbors, metric='cosine', algorithm="brute", n_jobs=-1)
    neighbors_model.fit(V_content)
    dists, indices = neighbors_model.kneighbors(V_topic)
    
    return indices

indices = get_matches(V_answer, V_misconception, n_neighbors=33)

np.save("indices.npy", indices)
df.to_parquet("df.parquet", index=False)

Writing run_recall.py


In [4]:
!python run_recall.py

Batches: 100%|████████████████████████████████████| 3/3 [00:01<00:00,  1.52it/s]
(40, 5120)


In [5]:
%%writefile run_reranker.py

from tqdm.auto import tqdm
import gc
import pandas as pd
import pickle
import sys
import os
import numpy as np
from tqdm.autonotebook import trange
from sklearn.model_selection import GroupKFold
import json
import torch
from numpy.linalg import norm
import torch.nn.functional as F
from torch import Tensor
from transformers import AutoTokenizer, AutoModel,BitsAndBytesConfig, AutoModelForCausalLM
from peft import (
    LoraConfig,
    get_peft_model,
    PeftModel,
)
import json
import copy
import warnings
warnings.filterwarnings('ignore')

rerank_model_path = "/kaggle/input/qwenqwen2.5-7b-instruct/pytorch/qwen2.5-14b-instruct/1"
rerank_lora_name_or_path = "/kaggle/input/rerank-v3-round2/rerank-v3-round2-checkpoint-654"
bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16
        )
tokenizer_0 = AutoTokenizer.from_pretrained(rerank_lora_name_or_path)
tokenizer_1 = AutoTokenizer.from_pretrained(rerank_lora_name_or_path)
tokenizer_0.bos_token_id = 151643
tokenizer_1.bos_token_id = 151643

device_0 = torch.device('cuda:0')  
model_0 = AutoModelForCausalLM.from_pretrained(rerank_model_path, device_map=device_0,use_cache=False,quantization_config=bnb_config,)
model_0 = PeftModel.from_pretrained(model_0, rerank_lora_name_or_path)
# model_0 = model_0.merge_and_unload()

device_1 = torch.device('cuda:1')  
model_1 = AutoModelForCausalLM.from_pretrained(rerank_model_path, device_map=device_1, use_cache=False,quantization_config=bnb_config,)
model_1 = PeftModel.from_pretrained(model_1, rerank_lora_name_or_path)
model_0 = model_0.eval()
model_1 = model_1.eval()


def last_logit_pool(logits: Tensor,
                    attention_mask: Tensor) -> Tensor:
    left_padding = (attention_mask[:, -1].sum() == attention_mask.shape[0])
    if left_padding:
        return logits[:, -1, :]
    else:
        sequence_lengths = attention_mask.sum(dim=1) - 1
        batch_size = logits.shape[0]
        return torch.stack([logits[i, sequence_lengths[i], :] for i in range(batch_size)], dim=0)

def get_inputs(pairs, tokenizer, prompt=None, max_length=512):
    if prompt is None:
        prompt = "Given a math question and a misconcepte incorrect answer,determine whether the misconception is pertinent to the query by providing a prediction of either 'Yes' or 'No'."
    sep = "\n"
    prompt_inputs = tokenizer(prompt,
                              return_tensors=None,
                              add_special_tokens=False)['input_ids']
    sep_inputs = tokenizer(sep,
                           return_tensors=None,
                           add_special_tokens=False)['input_ids']
    inputs = []
    for query, passage in pairs:
        query = preprocess_text(query)
        query_inputs = tokenizer(f'A: {query}',
                                 return_tensors=None,
                                 add_special_tokens=False,
                                 max_length=max_length * 3 // 4,
                                 truncation=True)
        passage_inputs = tokenizer(f'B: {passage}',
                                   return_tensors=None,
                                   add_special_tokens=False,
                                   max_length=max_length // 4,
                                   truncation=True)
        item = tokenizer.prepare_for_model(
            query_inputs['input_ids'],
            sep_inputs + passage_inputs['input_ids'],
            truncation='only_second',
            max_length=max_length,
            padding=False,
            return_attention_mask=False,
            return_token_type_ids=False,
            add_special_tokens=False
        )
        item['input_ids'] = item['input_ids'] + sep_inputs + prompt_inputs
        item['attention_mask'] = [1] * len(item['input_ids'])
        inputs.append(item)

    return tokenizer.pad(
            inputs,
            padding=True,
            max_length=max_length + len(sep_inputs) + len(prompt_inputs),
            pad_to_multiple_of=8,
            return_tensors='pt',
    )

import re
def preprocess_text(x):
    x = re.sub("http\w+", '',x)   # Delete URL
    x = re.sub(r"\.+", ".", x)    # Replace consecutive commas and periods with one comma and period character
    x = re.sub(r"\,+", ",", x)
    x = re.sub(r"\\\(", " ", x)
    x = re.sub(r"\\\)", " ", x)
    x = re.sub(r"[ ]{1,}", " ", x)
    x = x.strip()                 # Remove empty characters at the beginning and end
    return x

@torch.no_grad()
@torch.cuda.amp.autocast()
def inference_reranker(df, model, tokenizer, device, batch_size=4):
    all_scores = []
    # tokenizer = AutoTokenizer.from_pretrained(rerank_lora_name_or_path)
    inputs = get_inputs(list(df['predict_text'].values), tokenizer)
    yes_loc = tokenizer('Yes', add_special_tokens=False)['input_ids'][0]
    for i in trange(0, len(inputs['input_ids']), batch_size, desc="Batches", disable=False):
        batch_inputs = inputs[i:i+batch_size]
        batch_inputs = batch_to_device(batch_inputs, device)
        outputs = model(**batch_inputs, output_hidden_states=True)
        logits = outputs.logits
        scores = last_logit_pool(logits, batch_inputs['attention_mask'])
        scores = scores[:, yes_loc]
        all_scores.extend(scores.cpu().float().tolist())

    df['scores'] = all_scores
    return df



path_prefix = "/kaggle/input/eedi-mining-misconceptions-in-mathematics"

train_df = pd.read_parquet("df.parquet")
indices = np.load("indices.npy")

train_df['recall_ids'] = indices.tolist()

misconception_mapping = pd.read_csv(f"{path_prefix}/misconception_mapping.csv")

misconception_mapping['query_text'] = misconception_mapping['MisconceptionName']
misconception_mapping['order_index'] = misconception_mapping['MisconceptionId']

misconception_mapping_dict = {}
for _, row in misconception_mapping.iterrows():
    misconception_mapping_dict[row['order_index']] = row['query_text']

TOP_NUM = 25
df_rerank = []
for _,row in train_df.iterrows():
    for mis_id in row['recall_ids'][:TOP_NUM]:
        text = misconception_mapping_dict[mis_id]
        df_rerank.append({
            "order_index": row['order_index'],
            'predict_text': [row['query_text'], text],
            'question_id': row['QuestionId'],
            'mis_id': mis_id
        })
df_rerank = pd.DataFrame(df_rerank)


from concurrent.futures import ThreadPoolExecutor
df_rerank['predict_text_len'] =  df_rerank['predict_text'].apply(lambda x: len("".join(x)))
df_rerank = df_rerank.sort_values("predict_text_len", ascending=False)

df_rerank_1 = df_rerank.iloc[0::2].copy()
df_rerank_2 = df_rerank.iloc[1::2].copy()

with ThreadPoolExecutor(max_workers=2) as executor:
    rerank_results = executor.map(inference_reranker, (df_rerank_1, df_rerank_2), (model_0, model_1), (tokenizer_0, tokenizer_1), (device_0, device_1))
df_rerank = pd.concat(list(rerank_results), axis=0)

rerank_predicts_test = []
for _, df_g in df_rerank.groupby("order_index"):
    scores = df_g['scores'].values
    score_indexs = np.argsort(-scores)
    mis_ids = df_g['mis_id'].values
    rerank_predicts_test.append([mis_ids[index] for index in score_indexs[:TOP_NUM]])
train_df['rerank_ids'] = rerank_predicts_test

final_predict = []
for i in range(len(predicts_test)):
    temp = rerank_predicts_test[i] + predicts_test[i][TOP_NUM:]
    final_predict.append(temp)

train_df['rerank_ids'] = final_predict

indices = np.array(final_predict)
np.save("indices.npy", indices)
train_df.to_parquet("df.parquet", index=False)

Writing run_reranker.py


In [6]:
# !python run_reranker.py

In [7]:
import time
time.sleep(10)

In [8]:
%%writefile run_vllm.py

import vllm
import numpy as np
import pandas as pd
from transformers import PreTrainedTokenizer, AutoTokenizer
from typing import List
import torch
from logits_processor_zoo.vllm import MultipleChoiceLogitsProcessor
import re
import math
import os
# os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

model_path = "/kaggle/input/72b-gptq-cpt400/pytorch/default/1"
tokenizer = AutoTokenizer.from_pretrained(model_path)


def preprocess_text(x):
    x = re.sub("http\w+", '',x)   # Delete URL
    x = re.sub(r"\.+", ".", x)    # Replace consecutive commas and periods with one comma and period character
    x = re.sub(r"\,+", ",", x)
    x = re.sub(r"\\\(", " ", x)
    x = re.sub(r"\\\)", " ", x)
    x = re.sub(r"[ ]{1,}", " ", x)
    x = x.strip()                 # Remove empty characters at the beginning and end
    return x

PROMPT  = """
You are an expert math tutor who knows about all grade-school level math misconceptions.Your task is to diagnose the accurate type of misconceptions your student
have based on the (incorrect) answer he/she gives.Your selected misconception type should correspond to the given question and incorrect answer.

ConstructName: {ConstructName}({SubjectName}).
Question: {Question}
Correct Answer: {CorrectAnswer}
Incorrect Answer: {IncorrectAnswer}

Pick the misconception number from the below:
{Retrival}
"""
# just directly give your answers.

def apply_template(row, tokenizer):
    messages = [
        {
            "role": "user", 
            "content": preprocess_text(
                PROMPT.format(
                    ConstructName=row["ConstructName"],
                    SubjectName=row["SubjectName"],
                    Question=row["QuestionText"],
                    IncorrectAnswer=row[f"incorrect_answer"],
                    CorrectAnswer=row[f"correct_answer"],
                    Retrival=row[f"retrieval"]
                )
            )
        }
    ]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return text


misconception_df = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/misconception_mapping.csv")

df = pd.read_parquet("df.parquet")
indices = np.load("indices.npy")

# model_path = "/kaggle/input/72b-gptq-cpt400/pytorch/default/1"

# llm = vllm.LLM(
#     model_path,
#     quantization="gptq",
#     tensor_parallel_size=2,
#     gpu_memory_utilization=0.90, 
#     trust_remote_code=True,
#     dtype="half", 
#     enforce_eager=True,
#     max_model_len=5120,
#     disable_log_stats=True
# )
llm = vllm.LLM(
    model_path,
    quantization="gptq",
    tensor_parallel_size=2,
    gpu_memory_utilization=0.98, 
    trust_remote_code=True,
    dtype="half",
    enforce_eager=True,
    max_model_len=1758,
    disable_log_stats=True,
    cpu_offload_gb=8,
    swap_space=1,
    device='cuda',
    max_num_seqs=18
)
tokenizer = llm.get_tokenizer()

choices = ["1", "2", "3", "4", "5", "6", "7", "8", "9"]
KEEP = []
for x in choices:
    c = tokenizer.encode(x,add_special_tokens=False)[0]
    KEEP.append(c)

def get_candidates(c_indices):
    candidates = []

    mis_names = misconception_df["MisconceptionName"].values
    for ix in c_indices:
        c_names = []
        for i, name in enumerate(mis_names[ix]):
            c_names.append(f"{i+1}. {name}")

        candidates.append("\n".join(c_names))
        
    return candidates



def get_logits_scores(responses):
    results = []
    errors = 0
    for i,response in enumerate(responses):
        try:
            x = response.outputs[0].logprobs[0]
            logprobs = []
            for k in KEEP:
                if k in x:
                    logprobs.append( math.exp(x[k].logprob) )
                else:
                    logprobs.append( 0 )
                    print(f"bad logits {i}")
            logprobs = np.array( logprobs )
            logprobs /= logprobs.sum()
            results.append( logprobs )
        except:
            #print(f"error {i}")
            results.append( np.array(list(range(9,0,-1))) )
            errors += 1
            
    print(f"There were {errors} inference errors out of {i+1} inferences")
    results = np.vstack(results)
    results = np.argsort(-results)
    return results


survivors = indices[:, -1:]

for i in range(4):
    c_indices = np.concatenate([indices[:, -8*(i+1)-1:-8*i-1], survivors], axis=1)
    
    df["retrieval"] = get_candidates(c_indices)
    df["text"] = df.apply(lambda row: apply_template(row, tokenizer), axis=1)
    
    # print("Example:")
    # print(df["text"].values[0])
    # print()
    
    responses = llm.generate(
        df["text"].values,
        vllm.SamplingParams(
            n=1,  # Number of output sequences to return for each prompt.
            top_k=1,  # Float that controls the cumulative probability of the top tokens to consider.
            temperature=0,  # randomness of the sampling
            seed=777, # Seed for reprodicibility
            skip_special_tokens=False,  # Whether to skip special tokens in the output.
            max_tokens=1,  # Maximum number of tokens to generate per output sequence.
            logits_processors=[MultipleChoiceLogitsProcessor(tokenizer, choices=["1", "2", "3", "4", "5", "6", "7", "8", "9"])],
            logprobs = 13
        ),
        use_tqdm=True
    )

    # responses = [x.outputs[0].text for x in responses]
    # df["response"] = responses
    # llm_choices = df["response"].astype(int).values - 1
    # survivors = np.array([cix[best] for best, cix in zip(llm_choices, c_indices)]).reshape(-1, 1)

    sub_scores = get_logits_scores(responses)
    sub_indices = []
    for idx, x in enumerate(c_indices):
        scores_idx = sub_scores[idx]
        temp_list = []
        for s in scores_idx:
            temp_list.append(x[s])
        sub_indices.append(temp_list)
    
    sub_indices = np.array(sub_indices)
    
    indices[:, -8*(i+1)-1:-8*i-1] = sub_indices[:,1:]
    
    llm_choices = sub_indices[:, 0].tolist()
    survivors = sub_indices[:, 0].reshape(-1, 1)



results = []

for i in range(indices.shape[0]):
    ix = indices[i][:25]
    llm_choice = survivors[i, 0]
    
    results.append(" ".join([str(llm_choice)] + [str(x) for x in ix if x != llm_choice]))


df["MisconceptionId"] = results
df.to_csv("submission.csv", columns=["QuestionId_Answer", "MisconceptionId"], index=False)


Writing run_vllm.py


In [9]:
!python run_vllm.py

WARNING 12-09 16:20:28 cuda.py:22] You are using a deprecated `pynvml` package. Please install `nvidia-ml-py` instead, and make sure to uninstall `pynvml`. When both of them are installed, `pynvml` will take precedence and cause errors. See https://pypi.org/project/pynvml for more information.
WARNING 12-09 16:20:37 config.py:1668] Casting torch.bfloat16 to torch.float16.
WARNING 12-09 16:20:46 config.py:321] gptq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 12-09 16:20:46 config.py:905] Defaulting to use mp for distributed inference
WARNING 12-09 16:20:46 config.py:395] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 12-09 16:20:46 llm_engine.py:237] Initializing an LLM engine (v0.6.3.post1) with config: model='/kaggle/input/72b-gptq-cpt400/pytorch/default/1', speculative_config=None, tokenizer='/kaggle/input/72b-gptq-cpt400/pytorch/default/1',

In [10]:
import pandas as pd
sub = pd.read_csv("submission.csv")
sub.head()

,QuestionId_Answer,MisconceptionId
0,1869_B,15 38 10 22 27 36 16 29 35 30 19 18 26 17 31 0...
1,1869_C,14 38 23 22 7 31 35 18 16 19 29 27 12 36 10 26...
2,1869_D,15 31 23 38 22 18 35 36 10 4 16 29 21 30 26 27...
3,1870_A,3 0 38 24 36 37 13 25 21 15 32 16 30 23 5 31 2...
4,1870_B,3 31 13 38 37 25 36 0 22 7 20 23 35 32 5 21 16...
